<a href="https://colab.research.google.com/github/nawressabdel/Collecte-et-annotation-dataset/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Sep  1 08:20:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   Untitled0.ipynb   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 15490 (delta 10), reused 23 (delta 9), pack-reused 15460
Receiving objects: 100% (15490/15490), 14.10 MiB | 24.68 MiB/s, done.
Resolving deltas: 100% (10395/10395), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3-tiny.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3-tiny.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3-tiny.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3-tiny.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3-tiny.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3-tiny.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3-tiny.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3-tiny.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [ ]:
!echo "person" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-09-01 08:28:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  44.4MB/s    in 3.8s    

2022-09-01 08:28:22 (40.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: data/obj/fram34.jpg     
  inflating: data/obj/fram34.txt     
  inflating: data/obj/fram340.jpg    
  inflating: data/obj/fram340.txt    
  inflating: data/obj/fram341.jpg    
  inflating: data/obj/fram341.txt    
  inflating: data/obj/fram342.jpg    
  inflating: data/obj/fram342.txt    
  inflating: data/obj/fram343.jpg    
  inflating: data/obj/fram343.txt    
  inflating: data/obj/fram344.jpg    
  inflating: data/obj/fram344.txt    
  inflating: data/obj/fram345.jpg    
  inflating: data/obj/fram345.txt    
  inflating: data/obj/fram346.jpg    
  inflating: data/obj/fram346.txt    
  inflating: data/obj/fram347.jpg    
  inflating: data/obj/fram347.txt    
  inflating: data/obj/fram348.jpg    
  inflating: data/obj/fram348.txt    
  inflating: data/obj/fram349.jpg    
  inflating: data/obj/fram349.txt    
  inflating: data/obj/fram35.jpg     
  inflating: data/obj/fram35.txt     
  inflatin

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2142 data/obj/video-5An82wT7iBmfZSwx7-frame-006691-786M7fokQyoMzf6LG.txt
0 0.593750 0.475586 0.021875 0.080078
0 0.659375 0.471680 0.015625 0.076172

['0', '0.659375', '0.471680', '0.015625', '0.076172']
2142 data/obj/video-5An82wT7iBmfZSwx7-frame-006691-786M7fokQyoMzf6LG.txt
0 0.659375 0.471680 0.015625 0.076172
0 0.677344 0.473633 0.014063 0.068359

['0', '0.677344', '0.473633', '0.014063', '0.068359']
2142 data/obj/video-5An82wT7iBmfZSwx7-frame-006691-786M7fokQyoMzf6LG.txt
0 0.677344 0.473633 0.014063 0.068359
0 0.374219 0.469727 0.026562 0.099609

['0', '0.374219', '0.469727', '0.026562', '0.099609']
2143 data/obj/video-4FRnNpmSmwktFJKjg-frame-000891-WvoQdcroH4sjNmtRG.txt
0 0.374219 0.469727 0.026562 0.099609
0 0.213542 0.584722 0.256250 0.475000

['0', '0.213542', '0.584722', '0.256250', '0.475000']
2144 data/obj/fram105.txt
0 0.213542 0.584722 0.256250 0.475000
0 0.316667 0.430556 0.170833 0.255556

['0

IndexError: ignored

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/video-aMQG7QCtJKfnEdue5-frame-002960-8npeZmHDAfcmZntqj.jpg', 'data/obj/video-aMQG7QCtJKfnEdue5-frame-000982-rtHZmy29Xz5PfzL9Q.jpg', 'data/obj/video-nMfT5vK8MfEEjQ44W-frame-005011-DNCif43BTfc8EhTSM.jpg', 'data/obj/video-ZAtDSNuZZjkZFvMAo-frame-000407-7FqDgQidxQsopejdg.jpg', 'data/obj/video-nCxsLGvXK8o25AmXW-frame-002190-aj8Auu5e7JGBBSsT4.jpg', 'data/obj/fram36.jpg', 'data/obj/video-SCiKdG3MqZfiE292B-frame-007865-hh5j2LKoQJaFWdsEY.jpg', 'data/obj/video-JFiGa9oAAN2queuLt-frame-007521-tEpDGyRqWhmfkvPyY.jpg', 'data/obj/frame565.jpg', 'data/obj/video-SCiKdG3MqZfiE292B-frame-008056-TDw9gkL9Tf8DZAPo8.jpg', 'data/obj/video-AEZAaF8epmvQv5Nfj-frame-006620-gypbDkvEcxvCckNTK.jpg', 'data/obj/video-SCiKdG3MqZfiE292B-frame-007856-QZtHc2J2RN58nw4QA.jpg', 'data/obj/video-SCiKdG3MqZfiE292B-frame-007849-Y4ByKwdyjK5kNYjfM.jpg', 'data/obj/video-SCiKdG3MqZfiE292B-frame-006414-w3TJRXEDwwoCFhN7W.jpg', 'data/obj/video-5HPT2goT53ceRpvRm-frame-000630-PrZEg2W8BKCkEgyAn.jpg', 'data/obj/video-JhYLiFCieHQH

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg cfg/yolov3-tiny.conv.11  -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 total_bbox = 948058, rewritten_bbox = 2.439197 % 
]2;6199/500200: loss=0.8 hours left=137.1
 6199: 0.825605, 0.861873 avg loss, 0.000010 rate, 0.765081 seconds, 396736 images, 137.095827 hours left
Loaded: 0.000075 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.736903), count: 9, class_loss = 0.115305, iou_loss = 0.099567, total_loss = 0.214871 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.610334), count: 55, class_loss = 0.673666, iou_loss = 0.481322, total_loss = 1.154988 
 total_bbox = 948122, rewritten_bbox = 2.439138 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.760212), count: 19, class_loss = 0.214368, iou_loss = 0.112387, total_loss = 0.326755 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.651881), count: 41, class_loss = 0.553483, iou_loss = 0.2